In [9]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn, data_psrc
from collections import Counter

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2

df_pecarn = data_pecarn.get_data(use_processed=True, dummy=True)
df_psrc = data_psrc.get_data(use_processed=True, dummy=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
computing psrc preprocessing...


# compare rule to clinician suspicion
This only applies to pecarn

In [2]:
y = df['iai_intervention']
y1 = df['IAISuspicion'].values
y2 = df['IAIAcuteInterv'].values

In [3]:
print('IAISuspicion: Clinical suspicion for the presence of IAI identified on CT (split into 5 levels)')
print('score\tiaiw/i\ttot\tfrac')
for i in range(1, 7):
    num_iai = np.sum(y[y1==i])
    num_tot = np.size(y[y1==i])
    print(f'{i}\t {num_iai} \t {num_tot}\t{num_iai/num_tot:0.3f}')

IAISuspicion: Clinical suspicion for the presence of IAI identified on CT (split into 5 levels)
score	iaiw/i	tot	frac
1	 10 	 6591	0.002
2	 23 	 3124	0.007
3	 30 	 1166	0.026
4	 63 	 758	0.083
5	 77 	 298	0.258
6	 0 	 107	0.000


1. <1%
2. 1-5%
3. 6-10%
4. 11-50%
5. ">50%"
6. Physician did not answer

In [4]:
print('IAIAcuteInterv_1 - Clinical suspicion for IAI in need of acute intervention (split into 5 levels)')
print('score\tiaiw/i\ttot\tfrac')
for i in range(1, 7):
    num_iai = np.sum(y[y2==i])
    num_tot = np.size(y[y2==i])
    print(f'{i}\t {num_iai} \t {num_tot}\t{num_iai/num_tot:0.3f}')

IAIAcuteInterv_1 - Clinical suspicion for IAI in need of acute intervention (split into 5 levels)
score	iaiw/i	tot	frac
1	 35 	 9252	0.004
2	 40 	 1793	0.022
3	 33 	 506	0.065
4	 59 	 281	0.210
5	 36 	 87	0.414
6	 0 	 125	0.000


# counts should match pecarn paper

In [17]:
d = df_pecarn
ids_np = df.iai_intervention == 1
# print(list(d.keys()))

# the mean age is the only one which is wrong
print('iai', np.sum(d['iai']), np.sum(d['iai_intervention']))
print('age', np.mean(d.Age[ids_np]), np.mean(d.Age))
print('age<2', np.sum(d.Age[ids_np] < 2), np.sum(d.Age < 2))
print('sex', np.sum(d.Sex_M[ids_np]==1), np.sum(d.Sex_M==1))
print('hispanic ethnicity', np.sum(d.Race_Hispanic == 1), np.sum(d.Race_Hispanic))

iai 761 203
age 9.339901477832512 9.767270009963468
age<2 10 1167
sex 125 7384
hispanic ethnicity 1291 1291
